# Exploration 04
## 작사가 인공지능 만들기
---


In [1]:
import glob
import re
import os                                                                                                                                                                                                      
import numpy as np
import tensorflow as tf

In [2]:
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기:  187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


문장이 잘 불러와졌는지 출력해보자.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뛰기
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뛰기

    if idx > 9: break  
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


#### 텍스트 생성 모델을 만들기 위해서는 입력된 문장을 쪼개어 Tokenize 해야한다.
아래와 같은 기준으로 문장을 tokenize 해보자.
1. 소문자로 바꾸고, 양쪽 공백 지우기
2. 특수문자 양쪽에 공백을 넣고, 여러개의 공백은 하나의 공백으로 바꾸기.
3. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸고 다시 양쪽 공백을 지우기
4. 문장 시작에는 `<start>`, 끝에는 `<end>`를 추가하기.

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'\[[^)]*\]', '', sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'
    return sentence
print("[Test]")
print(preprocess_sentence("This @_is ;;;sample        sentence."))

[Test]
<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if preprocess_sentence(sentence) == '<start>  <end>':
        continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:20]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>']

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,
        filters=' ',
        oov_token="<unk>")
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2633 ...    0    0    0]
 [   2   35    7 ...   44    3    0]
 ...
 [   5   22    9 ...   10 1009    3]
 [  37   15 9032 ...  873  641    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4f51267ee0>


문장의 단어를 15개로 제한하였으므로 `maxlen=15` 코드를 `tensor`에 추가하였다.

In [7]:
print(tensor[:3])
tensor.shape

[[   2   50    5   91  296   64   57    9  963 6030    3    0    0    0
     0]
 [   2   17 2633  895    4    8   11 6031    6  329    3    0    0    0
     0]
 [   2   35    7   37   15  164  281   28  299    4   47    7   44    3
     0]]


(175406, 15)

In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])
src_input.shape

[   2   50    5   91  296   64   57    9  963 6030    3    0    0    0]
[  50    5   91  296   64   57    9  963 6030    3    0    0    0    0]


(175406, 14)

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 32)

print("shape of source train set: ", enc_train.shape)
print("shape of target train set: ", dec_train.shape)

shape of source train set:  (140324, 14)
shape of target train set:  (140324, 14)


총 데이터의 20% 를 평가 데이터 셋으로 사용하라고 하였으므로, 

In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
dataset_train

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 14
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, epochs=10, validation_data=dataset_val)

Epoch 1/10
548/548 [==============================] - 280s 477ms/step - loss: 3.7861 - val_loss: 3.3861
Epoch 2/10
548/548 [==============================] - 262s 477ms/step - loss: 3.2806 - val_loss: 3.2124
Epoch 3/10
548/548 [==============================] - 262s 478ms/step - loss: 3.0823 - val_loss: 3.0405
Epoch 4/10
548/548 [==============================] - 262s 478ms/step - loss: 2.8884 - val_loss: 2.9254
Epoch 5/10
548/548 [==============================] - 262s 477ms/step - loss: 2.7135 - val_loss: 2.8267
Epoch 6/10
548/548 [==============================] - 263s 479ms/step - loss: 2.5425 - val_loss: 2.7547
Epoch 7/10
548/548 [==============================] - 263s 479ms/step - loss: 2.3804 - val_loss: 2.6947
Epoch 8/10
548/548 [==============================] - 263s 479ms/step - loss: 2.2280 - val_loss: 2.6462
Epoch 9/10
548/548 [==============================] - 262s 478ms/step - loss: 2.0845 - val_loss: 2.6108
Epoch 10/10
548/548 [==============================] - 263s 479m

토큰의 개수가 15개가 되도록 하여 데이터의 shape이 (~,14)로 만들어졌다. 따라서 `embeding_size` 는 14로 설정하였다. <br>
hidden_size의 경우 512 부터 계속 바꿔가며 학습을 시켜보았는데 `hidden_size`가 2048일 때가 더 낮은 `val_loss`를 보여주었다.

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:

        predict = model(test_tensor) 
 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated    

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> she is", max_len=15)

'<start> she is a perfect illusion <end> '

# 회고

## 평가
노드에 나와있던 `i love` 로 시작하는 문장외에 `she go` 로 시작하는 문장으로 모델이 잘 학습되었는지를 확인한 결과, 각각 'i love you' 문장과 'she is a perfect illusion'의 문장이 출력된 것으로 보아 모델의 학습 자체는 잘 진행된 것으로 보인다. (두 번째 문장이 애매하긴 하다..)<br>
다만 데이터의 개수도 목표한 개수만큼 줄이지 못했고 목표했던 `val_loss`에는 끝내 도달하지 못했다. <Br>
`val_loss`가 목표치에 도달하지 못한 것이 적절한 하이퍼 파라미터값을 찾지 못한것, 그리고 데이터의 개수를 조금더 걸러내지 못했던 것이 원인이 아닐까 싶다.
<br> 특히 `hidden_size`의 값을 올릴 경우 모델의 학습속도가 굉장히 느려지는 것을 확인할 수 있어서 적절한 값을 찾는 것이 정확도 뿐만 아니라 효율성에서도 매우 중요할 것이다.

## 후기
최초로 진행한 NLP 프로젝트는 굉장히 낯설고 어렵게 느껴졌다. 이전에 했었던 CV 프로젝트에 비하여 직관적인 이해가 잘 되지 않았던 것 같다. 무엇보다 모델의 하이퍼 파라미터의 설정이 어려웠다. 노드 내용 외에 자료도 조금 더 찾아보았으나 이전에 공부했던 내용들에 비해 와닿지 않았던 것 같다. <br>
여러모로 배워야할 것이 많다는 게 실감나는 프로젝트였다. 자연어 처리에 대한 전반적인 이해가 많이 부족하다고 느꼈으며 더 공부를 해 가면서 다시 이 프로젝트를 보았을 때 어떠한 이유에 의하여 이러한 결과가 나타났는지, 또 어떻게 하면 성능을 더 끌어낼 수 있는 지에 대해서 다시 확인할 수 있도록 하겠다.

## 3. Reference
[1] https://algopoolja.tistory.com/34 
_embedding size 내용 참고_ <br>
[2] https://velog.io/@hwanython/%EC%9E%91%EC%82%AC%EA%B0%80-%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5-%EB%A7%8C%EB%93%A4%EA%B8%B0 _코드에 대한 설명 참고_